In [ ]:
#17. 이미지 다운로드용 웹크롤러 만들기
# Step 1. 필요한 모듈과 라이브러리를 로딩하고 검색어를 입력 받습니다
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import urllib.request
import urllib.parse
import time
import math
import os
import random

#Step 2. 사용자에게 검색 관련 정보들을 입력 받습니다.
print("=" *100)
print(" 이 크롤러는 휴넷 사이트의 강의 자료 수집용 웹크롤러입니다.")
print("=" *100)
query_txt = input('1.수집할 자료의 키워드는 무엇입니까?(예: 파이썬): ')

try :
    cnt = int( input('2.수집할 건수는 총 몇건입니까?(기본값:10): '))
except ValueError :
    cnt = 10
    print('기본값인 10 건으로 수집을 진행합니다.')
page_cnt = math.ceil( cnt / 12)

f_dir=input('3.파일이 저장될 경로만 쓰세요(예: c:\\py_temp\\ ) : ')
if f_dir =='' :
    f_dir = "c:\\py_temp\\"

#Step 3. 크롬 드라이버 설정 및 웹 페이지 열기
s = Service("c:/py_temp/chromedriver.exe")
driver = webdriver.Chrome(service=s)

url = 'https://www.hunet.co.kr/'
driver.get(url)
driver.maximize_window()
time.sleep(3)

#Step 4. 자동으로 검색어 입력 후 조회하기
element = driver.find_element(By.ID,'txtKeyword')
driver.find_element(By.ID,'txtKeyword').click( )
element.send_keys(query_txt)
element.send_keys("\n")

#Step 5. 사용자가 요청한 건수만큼 더보기 클릭하기
for a in range(0,page_cnt) :
    try :
        driver.find_element(By.XPATH,'//*[@id="divEducationList"]/div/div[2]/div[5]/a[1]').click()
        time.sleep(2)
        try :
            result = driver.switch_to_alert()
            result.accept( )
        except :
            continue
    except :
        print('페이지 이동이 끝났습니다. 이제 데이터를 수집하겠습니다')
        break
    
# Step 6. 이미지 추출하여 저장하기 
file_no = 0                                
count = 1
img_src2=[]  # 이미지 원본 URL 주소 저장할 리스트

n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)
img_dir = f_dir+s+'-'+query_txt
os.makedirs(img_dir)
os.chdir(img_dir)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
img_src = soup.find('ul','vod_list').find_all('img')

for i in img_src :
    img_src1=i['src']
    img_src2.append(img_src1)
    print(img_src1)
    count += 1
    if count > cnt :
        break

for i in range(0,len(img_src2)) :
    file_no += 1 
    urllib.request.urlretrieve(urllib.parse.quote(img_src2[i].encode('utf8'), '/:'),str(file_no)+'.jpg')

    time.sleep(0.5)      
    print("%s 번째 이미지 저장중입니다=======" %file_no)

# Step 7. 요약 정보를 출력합니다                
print("=" *70)
print("총 저장 건수는 %s 건 입니다 " %file_no)
print("파일 저장 경로: %s 입니다" %img_dir)
print("=" *70)

driver.close( )

In [ ]:
# Case 2. pixapay 사이트에서 그림 수집하기
# Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import urllib.request
import urllib
import time
import math
import os
import random

#Step 2. 필요한 정보를 입력 받습니다.
print("=" *80)
print(" pixabay 사이트에서 이미지를 검색하여 수집하는 크롤러 입니다 ")
print("=" *80)

query_txt = input('1.크롤링할 이미지의 키워드는 무엇입니까?: ')
cnt = int(input('2.크롤링 할 건수는 몇건입니까?: '))
real_cnt = math.ceil(cnt / 100) # 실제 크롤링 할 페이지 수
f_dir=input('3.파일이 저장될 경로만 쓰세요(예: c:\\py_temp\\ ) : ')
if f_dir =='' :
    f_dir = "c:\\py_temp\\"
    
print("\n")
print("요청하신 데이터를 수집 중이오니 잠시만 기다려 주세요~~^^")

#Step 3. 파일을 저장할 폴더를 생성합니다
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

img_dir = f_dir+s+'-'+query_txt
os.makedirs(img_dir)
os.chdir(img_dir)      

#Step 4. 크롬 드라이버를 사용해서 웹 브라우저를 실행한 후 검색합니다
s_time = time.time( )

s = Service("c:/py_temp/chromedriver.exe")
driver = webdriver.Chrome(service=s)

driver.get('https://pixabay.com/ko/')
time.sleep(3)

# 검색어 입력 창에 검색어 입력 후 검색 수행
element = driver.find_element(By.XPATH,'//*[@id="hero"]/div[4]/form/div/span/input')
element.send_keys(query_txt)
element.submit()

# Step 5. 이미지 추출하여 저장하기 
file_no = 1
count = 1
img_src2=[]    #이미지 파일의 url 주소 저장용 리스트

# 스크롤 다운 함수 만들기
def scroll_down(driver):
    #driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    driver.execute_script("window.scrollBy(0,1000)")
    time.sleep(1)

for a in range(1 , real_cnt+1):  
    
    for b in range(0,5):
        scroll_down(driver)
        time.sleep(1)
        
    # 원본 이미지 url 주소 수집 
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')    
    img_src = soup.find('div','row-masonry search-results').find_all('img')
   
    for c in img_src :
        img_src1=c['src']
        
        if 'http' in img_src1 :
            img_src2.append(img_src1)
            print(img_src1)
            
        count += 1  
        
        if count > cnt :
            break
            
    #수집된 url 주소로 이미지 파일 가져와서 저장하기
    for e in range(0,len(img_src2)) :

        class AppURLopener(urllib.request.FancyURLopener):
            version = "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) \
                       Chrome/47.0.2526.69 Safari/537.36"

        urllib.urlopener = AppURLopener()
        urllib.urlopener.retrieve(img_src2[e],str(file_no)+'.jpg')

        print("%s 페이지에서 %s 번째 이미지 저장중입니다=======" %(a,file_no))

        time.sleep(0.5) 

        if file_no >= cnt :
                break

        file_no += 1  
                
    if a > real_cnt :
        break

# Step 6. 요약 정보를 출력합니다                
e_time = time.time( )
t_time = e_time - s_time

print("=" *70)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("총 저장 건수는 %s 건 입니다 " %file_no)
print("파일 저장 경로: %s 입니다" %img_dir)
print("=" *70)

driver.close( )